# Data Cleanup and Transformation

## Open CSV data for cleanup

In [ ]:
import pandas as pd
df = pd.read_csv("location_of_csv_file")

In [ ]:
df.head()

## Extract year from date

Lets try with date of first row. Lets see all the fields of the first row using iloc

In [ ]:
df.iloc[0]

We can just extract the date field from the first row.

In [ ]:
??

We follow the instruction given in stackoverflow to convert excel date to python datetime

https://stackoverflow.com/questions/38454403/convert-excel-style-date-with-pandas

In [ ]:
import datetime as dt
dateval = pd.TimedeltaIndex([int(??)],unit='d') + dt.datetime(1899, 12, 30)
dateval

In [ ]:
??

Lets apply this to all the rows.

In [ ]:
pd.TimedeltaIndex([pd.to_numeric(df["register_date"])],unit='d') + dt.datetime(1899, 12, 30)

There are "2069-04-31" dates in the rows. Excel didn't convert these because this is invalid english date.

Lets identity the rows with such date "2069-04-31".

In [ ]:
df[df["register_date"].str.contains("-")==True]["register_date"].head()

Since this is string (object), we can now extract the first 4 char (year) from the above values.

In [ ]:
??

Now we can find the dates with numeric values.

In [ ]:
print(df[df["register_date"].str.contains("-")==False]["register_date"].head(3))

Now lets create a new column "register_year" following the above code.

In [ ]:
import datetime as dt
dateval = pd.TimedeltaIndex(pd.to_numeric(df[df["register_date"].str.contains("-")==False]["register_date"]), unit = "d")  + dt.datetime(1899, 12, 30)
df.loc[df["register_date"].str.contains("-")==False,"register_year"] = ??
df.loc[df["register_date"].str.contains("-")==True,"register_year"] = pd.to_numeric(df[df["register_date"].str.contains("-")==True]["register_date"].str[:4])
print(df.groupby("register_year").size())

## Check District names

Lets do quick summary of district names.

In [ ]:
df.groupby("district").size()

We don't see all the names. Lets display full list. 

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print_full(df.groupby("district").size())

All districts' names look ok here. There are few spelling mistakes but we can live with that for now.

## Check "Total Capital"

"Total Capital" should be of numeric type. Lets see the types of all the fields.

### Show field types

In [ ]:
df.dtypes

We see that total_capital is "object" type, which is not numeric. See https://pbpython.com/pandas_dtypes.html for different panda datatypes used.

### Converting "total capital" to numeric

In [ ]:
df["total_capital"] = pd.to_numeric(df["total_capital"])

There are ## characters in the "Total Capital" column. Lets see what are those. Lets see the rows containing ## characters in the "Total Capital"

In [ ]:
df[df["total_capital"].str.contains("##")].head(3)

These must have been issues when creating pdf from excel. For us, they are missing values. Either we could use 0 or employ other techniques to fill those missing values. For simplicity, lets fill those with 0.

In [ ]:
df.loc[df["total_capital"].str.contains("#"), "total_capital"] = 0

Now lets try converting to numeric type.

In [ ]:
df["total_capital"] = pd.to_numeric(df["total_capital"])

Lets calculate the sum of "total capital". We should get proper numeric value.

In [ ]:
df["total_capital"].sum()

## Check employment values

Lets see the type of "employment" column

In [ ]:
df["employment"].dtypes

It's int64, which means it's numeric column.

In [ ]:
df["employment"]

In [ ]:
df["employment"].sum()

## Check category

Category should be strings or object type. But we just need to make sure that there are no spelling mistakes.

In [ ]:
df["category"]

In [ ]:
grouped = df.groupby("category")
grouped["category"].count()

## Check scale

"Scale" is also of object type. Lets quickly check the summary of "Scale" values.

In [ ]:
df.groupby("scale").size()

## Split investment into local and foreign

"Investment" column contains the data in the following format
* Local - 100%
* Local - 64.11%, Foreign - 35.89%
* Foreign - 100%

We can't do anything with this column in its current form. But if we could separate local and foreign investment into 2 different columns, that could help us in various analysis. 

**Question for discussion**

* How would you approach this problem of splitting into 2 columns as shown above?

In [ ]:
df.investment

### New column for local and foreign percentage

Lets use regex to extract Local percentage value and create new column.

In [ ]:
df["local_percent"] = df.investment.str.extract("??")
df["local_percent"].fillna(0)
df.local_percent = pd.to_numeric(df.local_percent)

In [ ]:
df["foreign_percent"] = df.investment.str.extract("??")
df["foreign_percent"].fillna(0)
df.foreign_percent = pd.to_numeric(df.foreign_percent)

## Final data types of cleaned data

In [ ]:
df.dtypes

## Write to CSV

Lets write clean dataframe to file for exploration later.

In [ ]:
df.to_csv("data/clean.csv", index = False)

Now we will explore the data for sights.